<a href="https://colab.research.google.com/github/Rabinlamichhane1606/Data-Science/blob/master/GunsClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Model
import pandas as pd

# Set the paths for the dataset
train_data_dir = '/content/drive/MyDrive/Colab Notebooks/Guns_dataset/processed_dataset/train'

# Function to create a ResNet50 model
def create_resnet50_model(input_shape, num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(512, activation='relu')(x)
    output = layers.Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)

    # Freeze the layers of ResNet50 to prevent them from training initially
    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Prepare the data generator for rescaling
datagen = ImageDataGenerator(rescale=1./255)

# Load the dataset without shuffling, since we're doing it manually with KFold
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),  # ResNet50 requires 224x224 input size
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Extract file paths and labels
X = np.array(train_generator.filepaths)
y = np.array(train_generator.classes)

# Convert numerical labels to their string equivalents
class_indices = train_generator.class_indices
classes = {v: k for k, v in class_indices.items()}
y_str = np.array([classes[i] for i in y])

# Set up K-Fold Cross Validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Initialize variables to store results
fold_no = 1
acc_per_fold = []

# Perform K-Fold Cross-Validation
for train_index, val_index in kf.split(X):
    # Create the data generators for the current fold
    train_generator_fold = datagen.flow_from_dataframe(
        dataframe=pd.DataFrame({'filename': X[train_index], 'class': y_str[train_index]}),
        directory=None,
        x_col='filename',
        y_col='class',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        shuffle=True
    )

    val_generator_fold = datagen.flow_from_dataframe(
        dataframe=pd.DataFrame({'filename': X[val_index], 'class': y_str[val_index]}),
        directory=None,
        x_col='filename',
        y_col='class',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        shuffle=True
    )

    # Create a new ResNet50 model for each fold
    model = create_resnet50_model(input_shape=(224, 224, 3), num_classes=train_generator.num_classes)

    # Train the model
    print(f'Training for fold {fold_no} ...')
    history = model.fit(train_generator_fold, epochs=10, validation_data=val_generator_fold,
                        callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)])

    # Evaluate the model
    scores = model.evaluate(val_generator_fold)
    acc_per_fold.append(scores[1])
    print(f'Score for fold {fold_no}: {model.metrics_names[1]} of {scores[1] * 100:.2f}%')
    fold_no += 1

# Print the average accuracy
print(f'Average accuracy over {k} folds: {np.mean(acc_per_fold) * 100:.2f}%')


Found 177 images belonging to 3 classes.
Found 141 validated image filenames belonging to 3 classes.
Found 36 validated image filenames belonging to 3 classes.
Training for fold 1 ...
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 45s 8s/step - accuracy: 0.5362 - loss: 1.3757 - val_accuracy: 0.5556 - val_loss: 1.2038
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 34s 7s/step - accuracy: 0.5936 - loss: 1.0788 - val_accuracy: 0.5556 - val_loss: 1.3252
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 41s 7s/step - accuracy: 0.6628 - loss: 0.9104 - val_accuracy: 0.5556 - val_loss: 1.0380
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 45s 7s/step - accuracy: 0.6216 - loss: 0.8749 - val_accuracy: 0.5556 - val_loss: 1.0054
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 37s 7s/step - accuracy: 0.6606 - loss: 0.8377 - val_accuracy: 0.5556 - val_loss: 1.1519
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 41s 7s/step - accuracy: 0.6163 - loss: 0.9001 - val_accuracy: 0.5556 - val_loss: 0.9751
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 44s 7s/step - accuracy: 0.6841 - loss: 0.7736 - val_accuracy: 0.5556 - val_loss: 1.0796
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 33s 7s/step - accuracy: 0.7520 - loss: 0.6285 - val_accuracy: 0.4722 - val_loss: 1.0199
Epoch 9/10
5/5 ━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 45s 8s/step - accuracy: 0.5199 - loss: 1.1310 - val_accuracy: 0.5556 - val_loss: 1.0271
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 42s 9s/step - accuracy: 0.6124 - loss: 0.9942 - val_accuracy: 0.5556 - val_loss: 0.9773
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 37s 7s/step - accuracy: 0.6615 - loss: 0.8851 - val_accuracy: 0.5556 - val_loss: 0.9068
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 41s 7s/step - accuracy: 0.6735 - loss: 0.8261 - val_accuracy: 0.5556 - val_loss: 0.9002
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 41s 7s/step - accuracy: 0.6582 - loss: 0.8215 - val_accuracy: 0.5556 - val_loss: 0.8933
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 45s 7s/step - accuracy: 0.6793 - loss: 0.7448 - val_accuracy: 0.5833 - val_loss: 0.9014
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 33s 8s/step - accuracy: 0.7147 - loss: 0.7555 - val_accuracy: 0.5556 - val_loss: 0.8883
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 33s 7s/step - accuracy: 0.6535 - loss: 0.8059 - val_accuracy: 0.5556 - val_loss: 0.9288
Epoch 9/10
5/5 ━━━━

...

5/5 ━━━━━━━━━━━━━━━━━━━━ 44s 7s/step - accuracy: 0.4013 - loss: 1.3820 - val_accuracy: 0.1429 - val_loss: 1.3013
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 38s 8s/step - accuracy: 0.4037 - loss: 1.0874 - val_accuracy: 0.7143 - val_loss: 0.8571
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 41s 7s/step - accuracy: 0.5778 - loss: 1.1188 - val_accuracy: 0.6571 - val_loss: 0.8938
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 41s 7s/step - accuracy: 0.6280 - loss: 0.9170 - val_accuracy: 0.7143 - val_loss: 0.7512
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 42s 7s/step - accuracy: 0.6313 - loss: 0.8438 - val_accuracy: 0.7143 - val_loss: 0.7974
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 33s 7s/step - accuracy: 0.6350 - loss: 0.8922 - val_accuracy: 0.7143 - val_loss: 0.7216
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 42s 7s/step - accuracy: 0.5863 - loss: 0.8475 - val_accuracy: 0.7143 - val_loss: 0.7155
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 44s 8s/step - accuracy: 0.6432 - loss: 0.8103 - val_accuracy: 0.7143 - val_loss: 0.7426
Epoch 9/10
5/5 ━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 44s 7s/step - accuracy: 0.4859 - loss: 1.3309 - val_accuracy: 0.6286 - val_loss: 1.0253
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 34s 6s/step - accuracy: 0.6140 - loss: 1.0438 - val_accuracy: 0.6000 - val_loss: 0.9610
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 40s 7s/step - accuracy: 0.5668 - loss: 0.9396 - val_accuracy: 0.6286 - val_loss: 0.9422
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 34s 6s/step - accuracy: 0.6543 - loss: 0.8429 - val_accuracy: 0.6286 - val_loss: 0.9409
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 38s 7s/step - accuracy: 0.5817 - loss: 0.8762 - val_accuracy: 0.6286 - val_loss: 0.8887
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 36s 8s/step - accuracy: 0.6198 - loss: 0.8199 - val_accuracy: 0.6286 - val_loss: 0.8817
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 38s 7s/step - accuracy: 0.6768 - loss: 0.7852 - val_accuracy: 0.6571 - val_loss: 0.8775
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 37s 9s/step - accuracy: 0.6231 - loss: 0.8041 - val_accuracy: 0.6857 - val_loss: 0.8608
Epoch 9/10
5/5 ━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 45s 8s/step - accuracy: 0.4986 - loss: 1.2004 - val_accuracy: 0.6286 - val_loss: 0.8684
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 37s 7s/step - accuracy: 0.5962 - loss: 1.1124 - val_accuracy: 0.6286 - val_loss: 0.8154
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 42s 7s/step - accuracy: 0.6453 - loss: 0.9011 - val_accuracy: 0.6571 - val_loss: 0.9136
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 43s 8s/step - accuracy: 0.5862 - loss: 0.9256 - val_accuracy: 0.6571 - val_loss: 0.7815
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 43s 7s/step - accuracy: 0.5991 - loss: 0.9040 - val_accuracy: 0.6571 - val_loss: 0.7775
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 41s 8s/step - accuracy: 0.5911 - loss: 0.8577 - val_accuracy: 0.6571 - val_loss: 0.7636
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 33s 7s/step - accuracy: 0.6475 - loss: 0.7680 - val_accuracy: 0.6571 - val_loss: 0.7653
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 46s 7s/step - accuracy: 0.6682 - loss: 0.8273 - val_accuracy: 0.6571 - val_loss: 0.7324
Epoch 9/10
5/5 ━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
